In [7]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [8]:
df=pd.read_csv('final_data_for_modelling.csv')
df.head()

,gender,age,neighbourhood,scholarship,hypertension,diabetes,alcoholism,handicap,sms_received,showed,...,appointment_day_of_week,tempmax,temp,feelslikemax,feelslike,humidity,windspeed,solarradiation,solarenergy,uvindex
0,F,62,JARDIM DA PENHA,0,1,0,0,0,0,1,...,Friday,75.2,73.3,75.2,73.3,76.7,16.1,52.9,4.3,2.0
1,M,56,JARDIM DA PENHA,0,0,0,0,0,0,1,...,Friday,75.2,73.3,75.2,73.3,76.7,16.1,52.9,4.3,2.0
2,F,62,MATA DA PRAIA,0,0,0,0,0,0,1,...,Friday,75.2,73.3,75.2,73.3,76.7,16.1,52.9,4.3,2.0
3,F,8,PONTAL DE CAMBURI,0,0,0,0,0,0,1,...,Friday,75.2,73.3,75.2,73.3,76.7,16.1,52.9,4.3,2.0
4,F,56,JARDIM DA PENHA,0,1,1,0,0,0,1,...,Friday,75.2,73.3,75.2,73.3,76.7,16.1,52.9,4.3,2.0


In [9]:
y=df['showed']
X=df.drop('showed', axis=1)
X=X[['age','scholarship','hypertension','diabetes','alcoholism','handicap','sms_received','days_between_appointment_and_scheduled_day','tempmax','temp','feelslikemax','feelslike','humidity','windspeed','solarradiation','solarenergy','uvindex']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [11]:
model=RandomForestClassifier()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)

print(f'accuracy for test scores: {accuracy_score(y_test,y_pred):3f}')


accuracy for test scores: 0.772341
